In [1]:
import pandas as pd
import json

## Read all data files
The first column on Movies.csv is handled further manually due to encoding.

In [2]:
place_df = pd.read_csv('Harry_Potter_Movies/Places.csv')
chars_df = pd.read_csv('Harry_Potter_Movies/Characters.csv', encoding='ISO-8859-1')
dialogue_df = pd.read_csv('Harry_Potter_Movies/Dialogue.csv', encoding='ISO-8859-1')
chaps_df = pd.read_csv('Harry_Potter_Movies/Chapters.csv', encoding='ISO-8859-1')
movie_df = pd.read_csv('Harry_Potter_Movies/Movies.csv', encoding='ISO-8859-1').rename({'ï»¿Movie ID': 'Movie ID'}, axis='columns')

## Merge to a single dataframe
This helps us to extract either the ID or the actual name very easily.

In [3]:
df = pd.merge(dialogue_df, chaps_df, how="outer", on=["Chapter ID"])
df = pd.merge(df, chars_df, how="outer", on=["Character ID"])
df = pd.merge(df, place_df, how="outer", on=["Place ID"])
df = pd.merge(df, movie_df, how="outer", on=["Movie ID"])
df['Movie Title'] = df['Movie Title'].apply(lambda x: '_'.join(x.replace("'", '').lower().split()))
df[['Movie Title', 'Movie ID', 'Chapter ID', 'Chapter Name', 'Place ID', 'Place Name', 'Character Name']]

,Movie Title,Movie ID,Chapter ID,Chapter Name,Place ID,Place Name,Character Name
0,harry_potter_and_the_philosophers_stone,1,1,Doorstep Delivery,8,4 Privet Drive,Albus Dumbledore
1,harry_potter_and_the_philosophers_stone,1,1,Doorstep Delivery,8,4 Privet Drive,Albus Dumbledore
2,harry_potter_and_the_philosophers_stone,1,1,Doorstep Delivery,8,4 Privet Drive,Albus Dumbledore
3,harry_potter_and_the_philosophers_stone,1,1,Doorstep Delivery,8,4 Privet Drive,Albus Dumbledore
4,harry_potter_and_the_philosophers_stone,1,1,Doorstep Delivery,8,4 Privet Drive,Albus Dumbledore
...,...,...,...,...,...,...,...
7439,harry_potter_and_the_deathly_hallows_part_2,8,225,Your Mother's Eyes,24,Boathouse,Voldemort
7440,harry_potter_and_the_deathly_hallows_part_2,8,224,He Will Come To Me,24,Boathouse,Lucius Malfoy
7441,harry_potter_and_the_deathly_hallows_part_2,8,224,He Will Come To Me,24,Boathouse,Lucius Malfoy
7442,harry_potter_and_the_deathly_hallows_part_2,8,224,He Will Come To Me,24,Boathouse,Sybill Trelawney


## Combine to a dictionary
For each particular place and chapter we define all the characters contributing to the dialogues as a co-occurence network, unless it's just a single character at that space and time.

In [4]:
data_dict = {}
for movie_id in range(1, 9):
    temp_dict = {}
    temp = df[df['Movie ID'] == movie_id]
    title = list(set(temp['Movie Title']))
    assert len(title) == 1
    title = title[0]
    chaps, places = set(temp['Chapter Name']), set(temp['Place Name'])
    for chap_id in chaps:
        for place_id in places:
            temp2 = temp[(temp['Chapter Name'] == chap_id) & (temp['Place Name'] == place_id)]
            if temp2.shape[0] != 0:
                check = set(temp2['Character Name'])
                if len(check) != 1:
                    temp_dict[f'{chap_id} - {place_id}'] = ','.join(check)
    data_dict[title] = temp_dict

Wrap up the process by writing to a txt file in the harrypotter directory.

In [5]:
for title in data_dict:
    with open(f'harrypotter/{title}.txt', 'w') as f:
        for conn in data_dict[title].values():
            f.write(f'{conn}\n')
    f.close()

Export JSON string as well in case we need them.

In [6]:
json_string = json.dumps(data_dict)
with open('dialogues.json', 'w') as f:
    f.write(json_string)
f.close()

## Network processing

In [7]:
import seaborn as sns
import networkprocessing as np

In [8]:
# Read in the text file and process the network
np.make_network("harrypotter")

Processing harrypotter\harry_potter_and_the_chamber_of_secrets.txt
Processing harrypotter\harry_potter_and_the_deathly_hallows_part_1.txt
Processing harrypotter\harry_potter_and_the_deathly_hallows_part_2.txt
Processing harrypotter\harry_potter_and_the_goblet_of_fire.txt
Processing harrypotter\harry_potter_and_the_half-blood_prince.txt
Processing harrypotter\harry_potter_and_the_order_of_the_phoenix.txt
Processing harrypotter\harry_potter_and_the_philosophers_stone.txt
Processing harrypotter\harry_potter_and_the_prisoner_of_azkaban.txt


In [9]:
# Examine the dataset
networkInfo = pd.read_excel('harrypotter_analysis/networkInfo.xlsx')
nodeInfo = pd.read_excel('harrypotter_analysis/nodeInfo.xlsx')

In [10]:
networkInfo

,network_id,nodes,edges,avg path length,avg degree,avg weighted degree,diameter,radius,density
0,harry_potter_and_the_chamber_of_secrets,51,327,1.765490,12.823529,26.156863,3,2,0.256471
1,harry_potter_and_the_deathly_hallows_part_1,57,440,1.751880,15.438596,22.280702,3,2,0.275689
2,harry_potter_and_the_deathly_hallows_part_2,50,224,1.955918,8.960000,14.240000,4,2,0.182857
3,harry_potter_and_the_goblet_of_fire,42,265,1.772358,12.619048,25.761905,3,2,0.307782
4,harry_potter_and_the_half-blood_prince,39,213,1.805668,10.923077,20.871795,4,2,0.287449
5,harry_potter_and_the_order_of_the_phoenix,63,421,1.784434,13.365079,19.873016,2,1,0.215566
6,harry_potter_and_the_philosophers_stone,50,314,1.743673,12.560000,24.520000,2,1,0.256327
7,harry_potter_and_the_prisoner_of_azkaban,40,211,1.770513,10.550000,22.750000,3,2,0.270513


In [11]:
nodeInfo.iloc[:,1:]

,label,degree,weighted_degree,betweenness,normalized_betweenness,eccentricity,closeness,network
0,Minerva McGonagall,18,37,12.996032,0.010609,2,0.609756,harry_potter_and_the_chamber_of_secrets
1,Harry Potter,48,195,368.715704,0.300992,2,0.961538,harry_potter_and_the_chamber_of_secrets
2,Albus Dumbledore,21,49,20.449867,0.016694,2,0.632911,harry_potter_and_the_chamber_of_secrets
3,Serpent of Slitherin,14,21,1.445213,0.001180,2,0.581395,harry_potter_and_the_chamber_of_secrets
4,Poppy Pomfrey,14,22,1.445213,0.001180,2,0.581395,harry_potter_and_the_chamber_of_secrets
...,...,...,...,...,...,...,...,...
387,The Fat Lady,9,11,0.834524,0.001126,2,0.565217,harry_potter_and_the_prisoner_of_azkaban
388,Filius Flitwick,3,3,0.000000,0.000000,3,0.423913,harry_potter_and_the_prisoner_of_azkaban
389,Arthur Weasley,7,7,0.000000,0.000000,3,0.541667,harry_potter_and_the_prisoner_of_azkaban
390,Pixie,7,7,0.000000,0.000000,3,0.541667,harry_potter_and_the_prisoner_of_azkaban


In [12]:
for cols in ['degree', 'weighted_degree', 'betweenness', 'normalized_betweenness', 'eccentricity', 'closeness']:
    node_df = nodeInfo.sort_values(by=cols, ascending=False).iloc[:,1:]
    node_df.to_csv(f'harrypotter_analysis/{cols}.csv', index=False)

In [13]:
pd.read_csv('harrypotter_analysis/degree.csv')

,label,degree,weighted_degree,betweenness,normalized_betweenness,eccentricity,closeness,network
0,Harry Potter,62,165,919.999711,0.486515,1,1.000000,harry_potter_and_the_order_of_the_phoenix
1,Harry Potter,51,128,274.369995,0.178162,2,0.918033,harry_potter_and_the_deathly_hallows_part_1
2,Hermione Granger,51,120,282.233329,0.183268,2,0.918033,harry_potter_and_the_deathly_hallows_part_1
3,Ron Weasley,49,111,242.812999,0.157671,2,0.888889,harry_potter_and_the_deathly_hallows_part_1
4,Harry Potter,49,181,426.907251,0.363016,1,1.000000,harry_potter_and_the_philosophers_stone
...,...,...,...,...,...,...,...,...
387,Gang thug,1,1,0.000000,0.000000,4,0.382812,harry_potter_and_the_deathly_hallows_part_2
388,Dean Thomas,1,1,0.000000,0.000000,4,0.400000,harry_potter_and_the_half-blood_prince
389,Wizard,1,1,0.000000,0.000000,3,0.481013,harry_potter_and_the_half-blood_prince
390,Helena Ravenclaw,1,1,0.000000,0.000000,3,0.471154,harry_potter_and_the_deathly_hallows_part_2


In [14]:
pd.read_csv('harrypotter_analysis/weighted_degree.csv')

,label,degree,weighted_degree,betweenness,normalized_betweenness,eccentricity,closeness,network
0,Harry Potter,48,195,368.715704,0.300992,2,0.961538,harry_potter_and_the_chamber_of_secrets
1,Harry Potter,49,181,426.907251,0.363016,1,1.000000,harry_potter_and_the_philosophers_stone
2,Harry Potter,62,165,919.999711,0.486515,1,1.000000,harry_potter_and_the_order_of_the_phoenix
3,Ron Weasley,42,157,201.692868,0.164647,2,0.862069,harry_potter_and_the_chamber_of_secrets
4,Harry Potter,38,147,215.595779,0.262922,2,0.931818,harry_potter_and_the_goblet_of_fire
...,...,...,...,...,...,...,...,...
387,Diary,1,1,0.000000,0.000000,3,0.495050,harry_potter_and_the_chamber_of_secrets
388,Wizard,1,1,0.000000,0.000000,3,0.481013,harry_potter_and_the_half-blood_prince
389,Dean Thomas,1,1,0.000000,0.000000,4,0.400000,harry_potter_and_the_half-blood_prince
390,Gang thug,1,1,0.000000,0.000000,4,0.382812,harry_potter_and_the_deathly_hallows_part_2


In [15]:
pd.read_csv('harrypotter_analysis/betweenness.csv')

,label,degree,weighted_degree,betweenness,normalized_betweenness,eccentricity,closeness,network
0,Harry Potter,62,165,919.999711,0.486515,1,1.000000,harry_potter_and_the_order_of_the_phoenix
1,Harry Potter,42,108,465.739899,0.396037,2,0.875000,harry_potter_and_the_deathly_hallows_part_2
2,Harry Potter,49,181,426.907251,0.363016,1,1.000000,harry_potter_and_the_philosophers_stone
3,Harry Potter,48,195,368.715704,0.300992,2,0.961538,harry_potter_and_the_chamber_of_secrets
4,Hermione Granger,51,120,282.233329,0.183268,2,0.918033,harry_potter_and_the_deathly_hallows_part_1
...,...,...,...,...,...,...,...,...
387,Mrs. Cole,2,2,0.000000,0.000000,3,0.383838,harry_potter_and_the_half-blood_prince
388,Severus Snape,9,9,0.000000,0.000000,3,0.478632,harry_potter_and_the_deathly_hallows_part_1
389,Narcissa Malfoy,9,9,0.000000,0.000000,3,0.478632,harry_potter_and_the_deathly_hallows_part_1
390,Fenrir Greyback,5,5,0.000000,0.000000,3,0.509091,harry_potter_and_the_deathly_hallows_part_1


In [16]:
pd.read_csv('harrypotter_analysis/normalized_betweenness.csv')

,label,degree,weighted_degree,betweenness,normalized_betweenness,eccentricity,closeness,network
0,Harry Potter,62,165,919.999711,0.486515,1,1.000000,harry_potter_and_the_order_of_the_phoenix
1,Harry Potter,42,108,465.739899,0.396037,2,0.875000,harry_potter_and_the_deathly_hallows_part_2
2,Harry Potter,49,181,426.907251,0.363016,1,1.000000,harry_potter_and_the_philosophers_stone
3,Harry Potter,35,138,242.703909,0.327536,2,0.906977,harry_potter_and_the_prisoner_of_azkaban
4,Harry Potter,48,195,368.715704,0.300992,2,0.961538,harry_potter_and_the_chamber_of_secrets
...,...,...,...,...,...,...,...,...
387,Pansy Parkinson,9,9,0.000000,0.000000,3,0.542857,harry_potter_and_the_half-blood_prince
388,Arthur Weasley,6,6,0.000000,0.000000,3,0.520548,harry_potter_and_the_half-blood_prince
389,Remus Lupin,6,6,0.000000,0.000000,3,0.520548,harry_potter_and_the_half-blood_prince
390,Nymphadora Tonks,6,6,0.000000,0.000000,3,0.520548,harry_potter_and_the_half-blood_prince


In [17]:
pd.read_csv('harrypotter_analysis/eccentricity.csv')

,label,degree,weighted_degree,betweenness,normalized_betweenness,eccentricity,closeness,network
0,Dean Thomas,1,1,0.000000,0.000000,4,0.400000,harry_potter_and_the_half-blood_prince
1,Gang thug,1,1,0.000000,0.000000,4,0.382812,harry_potter_and_the_deathly_hallows_part_2
2,Petunia Dursley,3,3,0.000000,0.000000,4,0.379845,harry_potter_and_the_deathly_hallows_part_2
3,Narcissa Malfoy,2,2,0.000000,0.000000,4,0.413043,harry_potter_and_the_half-blood_prince
4,Tom,4,4,0.000000,0.000000,3,0.500000,harry_potter_and_the_prisoner_of_azkaban
...,...,...,...,...,...,...,...,...
387,Hermione Granger,28,92,71.090873,0.101125,2,0.791667,harry_potter_and_the_half-blood_prince
388,Harry Potter,34,130,192.694444,0.274103,2,0.904762,harry_potter_and_the_half-blood_prince
389,Lucius Malfoy,9,9,5.550866,0.006769,2,0.561644,harry_potter_and_the_goblet_of_fire
390,Harry Potter,62,165,919.999711,0.486515,1,1.000000,harry_potter_and_the_order_of_the_phoenix


In [18]:
pd.read_csv('harrypotter_analysis/closeness.csv')

,label,degree,weighted_degree,betweenness,normalized_betweenness,eccentricity,closeness,network
0,Harry Potter,62,165,919.999711,0.486515,1,1.000000,harry_potter_and_the_order_of_the_phoenix
1,Harry Potter,49,181,426.907251,0.363016,1,1.000000,harry_potter_and_the_philosophers_stone
2,Harry Potter,48,195,368.715704,0.300992,2,0.961538,harry_potter_and_the_chamber_of_secrets
3,Harry Potter,38,147,215.595779,0.262922,2,0.931818,harry_potter_and_the_goblet_of_fire
4,Harry Potter,51,128,274.369995,0.178162,2,0.918033,harry_potter_and_the_deathly_hallows_part_1
...,...,...,...,...,...,...,...,...
387,Mrs. Cole,2,2,0.000000,0.000000,3,0.383838,harry_potter_and_the_half-blood_prince
388,Gang thug,1,1,0.000000,0.000000,4,0.382812,harry_potter_and_the_deathly_hallows_part_2
389,Nymphadora Tonks,3,3,0.000000,0.000000,3,0.382812,harry_potter_and_the_deathly_hallows_part_2
390,Kingsley Shacklebolt,3,3,0.000000,0.000000,3,0.382812,harry_potter_and_the_deathly_hallows_part_2
